More tools for Analyzing Grid Connectivity: Substation/Bus tools
=======

(For background information, take a look at IntroGridConnectivityTools notebook first.)

We are building a **graph-like representation of the electrical grid**. One interesting, and apparently underused, feature of grids is the ability to assign high-voltage lines to different buses on a substation. In the `grid2op` framework, each substation consists of up to 2 buses that can be used to assign generators or loads (externals), or lines. 

Nominally, all lines and externals are assigned to the same bus. However, different bus configurations can result in dramatically different grid behavior, and can in theory be used to cure grid issues such as line overloads.

Bus Configurations: Cutting Through the Combinatorics
----------
Nominally, it would seem like the number of bus configurations for a substation with $n$ connected elements is $2^{n}$; namely, each element can be connected to bus 1 or bus 2. You can make a simple symmetry argument to reduce this by a factor of 2: exchanging the bus labels $1\leftrightarrow 2$ you leads to the same configuration. With this in mind, the number of configurations drops to $2^{n-1}$.

Furthermore, we consider it invalid if a single line is connected to a bus (which has some intuitive sense). This reduces the number of valid configurations further, to $2^{n-1}-n$ ($n>2$). Note that, with these rules in place, buses with 2 or 3 elements only have 1 configuration option, e.g. they are essentially static.

But there are additional ways that bus configurations become invalid (or double-counted) when lines are disconnected. For instance, if a line is disconnected, then it is ambiguous whether it would have been assigned to bus 1 or bus 2. (**It is important to note here that line disconnection accounting is performed at the line level, not at the bus level.**) Therefore a bus configuration 1001*0* and 1001*1* (where the bold element is disconnected) are degenerate.

In this next segment, we have several goals:
 - We need to define a **way to label a bus configuration** in a way that is unique.
 - We want to develop a check to determine if a certain bus configuration is **valid or invalid**.
 - Once we have these, we can determine **how many bus configurations are possible**, given some number of connected elements.
- We also need an accounting mechanism to keep track of **degenerate bus configurations** in case of disconnected elements.

To define a unique label, we represent a bus configuration with a bitset, e.g. 0b1010 for elements 0 and 2 connected to "bus 0", and lines 1 and 3 connected to "bus 1". (Note that the leading "0b" is meant to indicate in the code that this is a binary number.) A bitwise representation is useful because we can use efficient bitwise operations to determine bus configuration validity. (Note that we are abandoning the "bus 1" and "bus 2" notation, in favor of 0 and 1.)

The bitwise representation of bus configurations allows us to impose restrictions that dictate the validity of a given bus configuration, in order to enforce symmetry and degeneracy rules:
 - By convention, we require the leftmost bit to be connected to bus 1. This avoids double-counting cases with $0\leftrightarrow1$ symmetry. Therefore e.g. 0b1100 is labeled as valid, while the degenerate 0b0011 is called invalid for counting purposes.
 - We consider illegal the case where a single line is connected to a bus -- e.g. 0b1110 is invalid.
 - When a line is disconnected, the number of possible bus configurations for an "N-line bus" should reduce to the number of possible configurations for an "N-minus-one-line bus". To avoid double-counting identical bitsets in these cases, valid bitsets must have the disconnected line(s) assigned to "bus 1". In reality, of course, they are not connected at all, but we explicitly want to label as invalid the bitsets with the disconnected line(s) assigned to "bus 0" to keep the valid bus states unique.

A function for Implementing Bus Rules
------
We implement these rules with functions in a module, whose results are shown below:

In [ ]:
# This helper module is mine:
import BusTopologyHelpers

The function `IsValidBooleanBusState(flag,nbits)` checks whether a boolean bus state (flag) is valid (you must also pass the number of total externals (bits). Let's check some of our examples above:

In [ ]:
BOLD = '\033[1m'
END = '\033[0m'
print(BOLD+'0 <--> 1 Symmetry:'+END)
print('Leftmost bit is 1: 0b{:0{}b} is {}'.format(0b10001,5,BusTopologyHelpers.IsValidBooleanBusState(0b10001,5)))
print('Leftmost bit is 0: 0b{:0{}b} is {}'.format(0b00011,5,BusTopologyHelpers.IsValidBooleanBusState(0b00011,5)))
print()
print(BOLD+'Single-line-on-a-bus rule:'+END)
print('One line on a bus: 0b{:0{}b} is {}'.format(0b11110,5,BusTopologyHelpers.IsValidBooleanBusState(0b11110,5)))

Enumerating configurations in the $n$-external scenario
----------
We can check all possible configurations, given a number of externals $n$:

In [ ]:
# Try up to 9 externals
for n in range(1,10) :
    # Check all 2^n boolean bus states
    ValidBusStates = list(BusTopologyHelpers.IsValidBooleanBusState(i,n) for i in range(pow(2,n)))
    print('Valid configurations in {}-externals case:'.format(n),sum(ValidBusStates))

Disconnecting lines
---------
Finally, we want to be able to disconnect $m$ lines and check the number of remaining possible configurations, which should be equal to the number of configurations in the $n-m$ case. We do this using the `items_disconnected` argument of the `IsValidBooleanBusState` function (see below).

We try this with 8 externals (lines), three of which are disconnected, and try to confirm that only 11 valid configurations remain.

In [ ]:
items_disconnected = [0,1,2]
nEdges = 8
print('\nInvestigating case with {} lines, with these lines disconnected:'.format(nEdges),items_disconnected)
n = 0
for i in range(pow(2,nEdges)) :
    isValid = BusTopologyHelpers.IsValidBooleanBusState(i,nEdges,items_disconnected=items_disconnected)
    if not isValid :
        continue
    print('Flag 0b{:0{}b} is {}'.format(i,nEdges,'legal' if isValid else 'illegal'))
    n += 1
print('For {} edges, there are {} configurations that are legal,'.format(nEdges,n),
      'given these disconnected items:',items_disconnected)

Islanded Loads and Generators
--------
We also want to ensure that our machinery correctly excludes cases where loads and generators are not connected to the grid. The case where the external is not connected to any other line is already excluded; however, there is the additional case where 2+ externals are not connected to any lines, only to each other (and this is illegal).

In [ ]:
bus_topo = 0b11001
bus_topo_str = '0b{:0{}b}'.format(bus_topo,5)

i_gens,i_loads,i_disconnected = [1],[2],[0]
valid = BusTopologyHelpers.IsValidBooleanBusState(bus_topo,5,
                                                  i_gens=i_gens,
                                                  i_loads=i_loads,
                                                  items_disconnected=i_disconnected)
valid_str = 'valid' if valid else 'NOT valid'
print('{} is {} with externals on:'.format(bus_topo_str,valid_str),i_gens + i_loads,
      'and disconnected lines:',i_disconnected)

i_gens,i_loads,i_disconnected = [1],[3],[0]
valid = BusTopologyHelpers.IsValidBooleanBusState(bus_topo,5,
                                                  i_gens=i_gens,
                                                  i_loads=i_loads,
                                                  items_disconnected=i_disconnected)
valid_str = 'valid    ' if valid else 'NOT valid'
print('{} is {} with externals on:'.format(bus_topo_str,valid_str),i_gens + i_loads,
      'and disconnected lines:',i_disconnected)

The Substation Class
==========

We now introduce a simple Substation class that can help us manipulate the adjacency matrix representing the grid. Again, the goal is to have some simple code that can study grid connectivity without the overhead of an entire grid simulation package. Let's first load the `rte_case14_realistic` grid so that we can manipulate it in the following.

In [ ]:
import grid2op
env = grid2op.make(test=True)

from grid2op.PlotGrid import PlotMatplot
plot_helper = PlotMatplot(env.observation_space)
_ = plot_helper.plot_info(line_values=['line %d'%(el) for el in range(env.n_line)])

Now we build instances of each substation class, using a function to construct the substation class instances from the grid2op `environment` class. Then we will illustrate how each substation encodes the information from the grid.

In [ ]:
import Substation
sub_classes = Substation.BuildSubstations(env)

for sub in sub_classes :
    print('Sub {} has {} elements:'.format(sub.index,sub.nElements))
    print(' -',list(sub.elementIDs))

The Dewey Decimal System of Substations
-------
What are these cryptic numbers for each element? They constitute a nomenclature system for easily reading off everything you need to know about an element. Why? Because let's face it, it is too difficult to easily understand a substation using the `grid2op environment` class.

So how do you read it? Easy as **TEEE.SSS**:
 - **T**: The thousandths-place value is the **T**ype of object:
    - 1 for line origin
    - 2 for line extremity
    - 3 for load
    - 4 for generator
 - **EEE**: The next three digits are reserved for the index of the **E**lement (e.g. "line 0", see the grid image)
 - **.SSS**: The three digits after the decimal (for lines only) indicate which **S**ubstation the line links to.
 
A few examples:
 - 1002.003 is a *line origin*, referring to line number 2, and it connects this substation to *substation 3*.
 - 2003.004 is a *line extremity*, referring to line number 3, and it connects this substation to *substation 4*.
 - 3004 is a *load*, referring to load number 4.
 - 4005 is a *generator*, referring to generator number 5.

It is probably a little difficult to get used to, but I personally have found it useful. If you don't want to bother with it, then it's fine; you can do whatever you were used to doing before.

Precomputed Valid Bus Configurations
-------

One benefit of the Substation class is that it precomputes the list of all valid substation configurations, given how it was initialized, and saves it in a cache. This will speed up later computations that may check configuration validity many many times.

Note that this feature is demonstrated below for the case where all lines are connected, but the cache also contains precomputed valid substation configurations for cases when specific lines are disconnected.

In [ ]:
for sub in sub_classes :
    validBusStates = sub.GetValidBusStates()
    dotdotdot = '...' if len(validBusStates) > 5 else ''
    busStatesPrintout = list('0b{:0{}b}'.format(a,sub.nElements) for a in validBusStates[:5])
    print('Sub {:2d} has {:2d} valid states:'.format(sub.index,len(validBusStates)),busStatesPrintout,dotdotdot)

Manipulating an Adjacency Matrix using a Substation instance
-------
In the following, we demonstrate how a substation configuration can be applied to an adjacency matrix (class instance), in order to set up a new version of that adjacency matrix (which can then be studied). This is useful to study the properties of different substation configurations.

In [ ]:
import TopologyHelpers
import numpy as np

adj_instance = TopologyHelpers.AdjacencyMatrixClass(env,lineOnBits=0b11111111111111111111)
originalMatrix = np.copy(adj_instance.adjacency_matrix)

### Here is some code to check the effect of each config  
#for i in sub_classes[5].GetValidBusStates() :
#    sub_classes[5].ApplyBusConfig(i,adj_instance,verbose=True,doNotExecute=True)

sub_classes[5].ApplyBusConfig(0b101110,adj_instance,verbose=True)
sub_classes[3].ApplyBusConfig(0b101110,adj_instance,verbose=True)
sub_classes[8].ApplyBusConfig(0b11100,adj_instance,verbose=True)
sub_classes[8].ApplyBusConfig(0b11010,adj_instance,verbose=True)

diff_matrix = adj_instance.adjacency_matrix - originalMatrix
BOLD = '\033[1m'
END = '\033[0m'
print(BOLD+'\nDifference b/w original matrix and the new one: (0: connection turned off; 1: connection turned on)'+END)
print(adj_instance.GetMatrixHeader())
for i,row in enumerate(diff_matrix) :
    header = adj_instance.GetRowHeader(i)
    print(header+' '.join(('%d'%((a+1)/2) if a != 0 else '·') for a in list(row)))

Wrapping Up
=======

In the following notebook, we have demonstrated a new accounting method for unique bus state configurations using bitwise flags. We have introduced a new nomenclature system for quickly reading the contents of the substations, and we have illustrated how to apply bus configuration changes to an adjacency matrix.

All of this functionality will be critical toward studying the behavior of the grid in different configuration scenarios in a comprehensive and systematic way.